In [1]:
from spacecraft.satellite import SatelliteImplementation
from setup.initial_settings import SimulationSetupReader
from setup.two_line_element import TwoLineElementReader
from visualziations.visualizations import MatplotlibPlots
from visualziations.visualizations import PlotlyPlots
from spacecraft.sensors import MagnetometerImplementation, SunsensorImplementation, SensorFusionImplementation
import core.transformations as tr
import core.utilities as ut
from pathlib import Path
from visualziations.visualizations import LivePlotlyLine
import numpy as np
from core.logger import log, warn


# Live angular velocity chart
live_w = LivePlotlyLine(
    labels=["wx", "wy", "wz", "|w|"],
    title="Angular velocity (live)",
    xlabel="Time (s)",
    ylabel="deg/s",
    window=1000,          # rolling window in 'Time (s)' units
)

setup = SimulationSetupReader(Path('setup/initial_settings.json'))
tle = TwoLineElementReader(Path('setup/tle'))
magnetometer = MagnetometerImplementation(setup)
sunsensor = SunsensorImplementation(setup)
sensor_fusion = SensorFusionImplementation(setup, ['triad', 'quest', 'ekf'], tr.euler_xyz_to_quaternion(setup.euler_angles))

satellite = SatelliteImplementation(setup, tle, magnetometer, sunsensor, sensor_fusion)
quaternion_prev = satellite.quaternion.copy()

satellite.state_vector.reset()
satellite.state_vector.next_row()
ut.basic_state_vector(satellite)
ut.log_init_state(setup)

n_iter = setup.iterations_info["Stop"]
for x in range(setup.iterations_info["Start"], n_iter, setup.iterations_info["Step"]):
    satellite.manage_actuators_sensors_timing()
    satellite.update_iteration(x)
    satellite.apply_rotation()

    mag_field_sbf, mag_field_eci = satellite.magnetic_field
    sun_vector_sbf, sun_vector_eci = satellite.sun_vector

    if x % setup.iterations_info["LogInterval"] == 0:
        log(f"Iteration {x} of {n_iter}")

    satellite.fuse_sensors(
        [mag_field_sbf, sun_vector_sbf],
        [mag_field_eci, sun_vector_eci],
        quaternion_prev
    )
    
    satellite.manage_modes()
    satellite.apply_detumbling()
    satellite.apply_pointing()

    satellite.state_vector.next_row()
    ut.basic_state_vector(satellite)

    wx, wy, wz = map(float, satellite.angular_velocity)
    wmag = float(np.sqrt(wx*wx + wy*wy + wz*wz))
    live_w.update(float(x), [wx, wy, wz, wmag])

    quaternion_prev = satellite.quaternion.copy() 

satellite.state_vector.to_csv('simulation_state_vector.csv')
mpl = MatplotlibPlots()
# renderer="vscode" or "browser"
pp = PlotlyPlots(save=True, show=False)
df = satellite.state_vector.to_dataframe()
mpl.basic_plots(df, setup)
pp.basic_plots(df, setup)


FigureWidget({
    'data': [{'mode': 'lines', 'name': 'wx', 'type': 'scatter', 'uid': '9d129292-ac97-4748-b0c7-2d11f7f04dc6'},
             {'mode': 'lines', 'name': 'wy', 'type': 'scatter', 'uid': 'c2102087-a09f-4de5-86ee-b217f1d9e49d'},
             {'mode': 'lines', 'name': 'wz', 'type': 'scatter', 'uid': '9591b2fa-26fe-4557-a019-4d1838031762'},
             {'mode': 'lines', 'name': '|w|', 'type': 'scatter', 'uid': '80f11baf-9096-44ba-8ac1-fea154028f20'}],
    'layout': {'template': '...',
               'title': {'text': 'Angular velocity (live)'},
               'xaxis': {'title': {'text': 'Time (s)'}},
               'yaxis': {'title': {'text': 'deg/s'}}}
})

2025-11-13 21:57:00 | INFO | Simulation initialized with the following parameters:
2025-11-13 21:57:00 | INFO | Number of iterations: 6000
2025-11-13 21:57:00 | INFO | Satellite mass: 1.2 kg
2025-11-13 21:57:00 | INFO | Satellite inertia: [[0.002 0.    0.   ]
 [0.    0.002 0.   ]
 [0.    0.    0.002]] kg*m^2
2025-11-13 21:57:00 | INFO | Initial angular velocity: [ 2. -3.  4.] deg/s
2025-11-13 21:57:00 | INFO | Initial attitude (Euler angles): [0. 0. 0.] deg
2025-11-13 21:57:00 | INFO | Selected sensor fusion algorithm: EKF
2025-11-13 21:57:00 | INFO | Magnetometer noise: 10.0 nT
2025-11-13 21:57:00 | INFO | Sunsensor noise: 0.2 degrees
2025-11-13 21:57:00 | INFO | Sensor on time: 2 seconds, actuator on time: 8 seconds 

2025-11-13 21:57:00 | INFO | Iteration 0 of 6000
2025-11-13 21:57:05 | INFO | Iteration 100 of 6000
2025-11-13 21:57:10 | INFO | Iteration 200 of 6000
2025-11-13 21:57:15 | INFO | Iteration 300 of 6000
2025-11-13 21:57:20 | INFO | Iteration 400 of 6000
2025-11-13 21:57:

In [ ]:
# TODO
# complete b-cross
# some optimization
# verify the descriptions and documentation
# add print with starting parameters
# implement logger

# OPTIONAL
# add bang-bang mode for b-dot
# add more sensors
# add more actuators
# tests for main parameters


Documentation:

1. Whats in this code

2. Short intro
    what is adcs
    simulating environment
    simulating sensors and actuartors
    basic control

3. Threoretical background
    simlating envireonment (tle, magnetic field, sun position etc)
    reference frames and transformations
    sensors (magnetometer, sun sensor, star tracker, gyroscope)


In [ ]:
import numpy as np
#                      x                 y                   z              x               y
array1 = np.array([-0.0114, -        -0.0114,          0.0114]) #         -0.0114, -        0.0114
array2 = np.array([ 1.92709051e-05, -3.48258106e-05,  1.31687787e-05]) # 1.92709051e-05, -3.48258106e-05

array3 = [
    (array1[1]*array2[2] - array1[2]*array2[1]),
    (array1[2]*array2[0] - array1[0]*array2[2]),
    (array1[0]*array2[1] - array1[1]*array2[0])
]

array3_np = np.cross(array1, array2)    

print(array3)
print(array3_np)
